## Web scraping(웹 스크래핑, 문서 수집하기)
[Open In Colab](https://colab.research.google.com/github/langchain-ai/langchain/blob/v0.1/docs/docs/use_cases/web_scraping.ipynb)

### 4. Research automation
* 구글 검색을 활용한 리서치 자동화
  * [Programmable Search Engine](https://cse.google.com/cse?cx=a61c425e10bee4b8d)
* APIFY를 활용한 리서치 자동화
  * [Integration Tutorial: How to use LangChain with Apify scrapers](https://www.youtube.com/watch?v=zcfeiVdiGJg)

#### 구성도
<img src="./images/web_research.png" width="800">

- [langchain-ai/web-explorer](https://github.com/langchain-ai/web-explorer) 예시 화면

<img src="./images/research_automation01.png" width="600"> <img src="./images/research_automation02.png" width="522">


#### Resources
 - [Custom Search JSON API: 소개](https://developers.google.com/custom-search/v1/introduction?hl=k")



In [1]:
from python_environment_check import check_packages

lib = {
    'streamlit': '1.29.0', 
    'langchain': '0.0.354',
    'chromadb': '0.4.3', 
    'openap': '1.30.1',
    'langchain-openai': '0.1.7',
    'langchain-core': '0.2.0',
    'langchain-chroma': '0.1.1',
    'openai': '1.30.1',
    'google-api-core': '2.11.1',
    'google-api-python-client': '2.95.0',
    'google-auth': '2.22.0',
    'google-auth-httplib2': '0.1.0',
    'googleapis-common-protos': '1.59.1',
    'tiktoken': '0.7.0',
    'faiss-cpu': '1.8.0',
    'apify-client': '1.7.0', 
#    'beautifulsoup4': '4.12.2',
    'html2text': '2024.2.26',
    'dotenv': '0.0.5',
    'nest_asyncio': '1.6.0',
}

#check_packages(lib)

[OK] Your Python version is 3.11.9 (main, May 28 2024, 08:05:11) [GCC 8.5.0 20210514 (Red Hat 8.5.0-20)]


In [5]:
import dotenv
dotenv.load_dotenv('../envls')

# 미리 실행해야 함
import nest_asyncio
nest_asyncio.apply()

# 미리 설치해야 함
#!playwright install


In [6]:
import os
#print(os.environ['GOOGLE_CSE_ID'])
#print(os.environ['GOOGLE_API_KEY'])

# 리눅스에서 sqlite3 인식
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


### 구글 검색을 활용한 리서치 자동화
#### 구글 Custom Search Engine 생성 및 키 필요
<img src="./images/googlecse01.png" width="970">
<img src="./images/googlecse02.png" width="800">
<img src="./images/googlecse03.png" width="980">
<img src="./images/googlecse04.png" width="800">
<img src="./images/googlecse05.png" width="800">
<img src="./images/googlecse06.png" width="880">
<img src="./images/googlecse07.png" width="980"> 

In [7]:
from langchain.retrievers.web_research import WebResearchRetriever
#import chromadb
#from langchain.indexes import VectorstoreIndexCreator
# from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



In [8]:
# Get the version of ChromaDB
# chroma_version = "0.4"
# chroma_version = chromadb.__version__
persist_directory = "./db/chroma_db_oai"

# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory=persist_directory
)

# LLM
llm = ChatOpenAI(temperature=0)

# Search
search = GoogleSearchAPIWrapper()

/home/www/.pyenv/versions/3.11.9/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [9]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [16]:
# Run
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.DEBUG)
from langchain.chains import RetrievalQAWithSourcesChain

#user_input = "How do LLM Powered Autonomous Agents work?"
#user_input = "고래대학교의 학생수는?"
# user_input = "고려대학교의 위치는 어디인가?"
user_input = "고려대학교의 학생수는 몇명인가?"


qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
result = qa_chain({"question": user_input})
result


Fetching pages: 100%|##########| 2/2 [00:00<00:00, 11.94it/s]


{'question': '고려대학교의 학생수는 몇명인가?',
 'answer': '고려대학교의 학생수는 서울캠퍼스에 30,320명이 있고, 세종캠퍼스에 7,501명이 있습니다. 따라서 총 재학생 수는 37,821명입니다.\n',
 'sources': 'https://www.korea.ac.kr/mbshome/mbs/university/subview.do?id=university_010106000000, https://ko.wikipedia.org/wiki/%EA%B3%A0%EB%A0%A4%EB%8C%80%ED%95%99%EA%B5%90'}

#### * APIFY를 활용한 리서치 자동화
#### [APIFY](https://apify.com/)
* 2015년에 설립된 프라하 기반 스타트업
* 대량의 웹 데이터를 수집 및 분석하고 웹 프로세스를 자동화하는 오픈 소스 도구 제공
* Apify Store를 운영하며, 필요한 스크래퍼를 등록하여 사용할 수 있음

* Apify 회원 가입 후 Apify Store에서 Actor id 저장
  * 해당 Actor의 scheme을 확인하여 검색하고자 하는 속성명 저장
  * 아래 1. 수집기를 활용한 방법에 적용한 후 실행
* Actor을 구동하면 Storage에 데이터셋이 저장되어 있음
  * 데이터셋 아이디가 있으면, 2. 수집한 데이터셋을 활용한 방법으로 진행해도 됨

<img src="./images/apify00.png" width="700">
<img src="./images/apify01.png" width="1050">
<img src="./images/apify05.png" width="980">
<img src="./images/apify02.png" width="800">
<img src="./images/apify03.png" width="800">
<img src="./images/apify04.png" width="850">

In [17]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.docstore.document import Document
from langchain_community.utilities import ApifyWrapper
from langchain_community.document_loaders import ApifyDatasetLoader

apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
"""
1. 수집기를 활용한 방법
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    #actor_id="astronomical_lizard/naver-blog-scraper", 
    #run_input={"startUrls": [{"url": "/docs/integrations/chat/"}]},
    run_input={"startUrls": [{"url": "https://docs.apify.com/academy/web-scraping-for-beginners"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["full_text"] or "", metadata={"source": item["url"]}
    ),
)
"""
# 2. 수집한 데이터셋을 활용한 방법
loader = ApifyDatasetLoader(
    #dataset_id="mCkg6zDcTEO5JYX2F", 
    dataset_id="rneQUybW8o0bUwZpK", 
    dataset_mapping_function=lambda item: Document(
        page_content=item["full_text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
index = VectorstoreIndexCreator(embedding=OpenAIEmbeddings()).from_loaders([loader])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature="0")

# Query the vector store
#query = "Are any OpenAI chat models integrated in LangChain?"
query = "Inspection에 대해서 알려주세요"
result = index.query(query, llm=llm)
print(result)


/home/www/.pyenv/versions/3.11.9/envs/langchain/lib/python3.11/site-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


Inspection은 제품, 시설, 문서 등을 세밀하게 조사하거나 검토하는 과정을 의미합니다. 이 과정은 일반적으로 제품의 품질이나 안전성을 확인하거나 규정에 따른 규정 준수 여부를 평가하기 위해 수행됩니다. Inspection은 주로 시각적으로 이루어지지만 필요에 따라 다양한 도구나 장비를 사용하여 수행될 수도 있습니다.


## 결론
- 서비스하기에는 속도가 느리다. (개선할 부분을 찾는 것이 숙제)
- 제공되는 Transformer의 세부적인 옵션이 필요하다.
  - BeautifulSoup 라이브러리를 사용하는 것이 더 효과적일 수 있다.
- 상용 검색엔진과 연동하면 더 효과적으로 서비스할 수 있을 것 같다.
- 스크래퍼의 변환 결과와 LLM의 스키마 및 프롬프트 결과를 위한 충분한 테스트 필요

